# Setup

In [140]:
import json
import tweepy
from tweepy import OAuthHandler
from pymongo import MongoClient
import os

In [141]:
# Twitter API setup
consumer_key = 'W1vEEpGJawjrqeW70yBUkfrWt'
consumer_secret = 'PSNivPYzJCjvEeQ5WW0zAhAIbAoEdzQBjTUG39up4yhnW9HABi'
access_token = '289683843-a9GiDa84Ak7FqD8c8Cu1qYzA5oEVqQ1FSKjkSudP'
access_secret = 'O3KhEusOxqTJGI6zvh8eqlDbyGyUT4tKThymbr57YFHWx'
 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth)

In [143]:
# MongoDB setup
os.system('mongo twitterdb --eval "db.dropDatabase();"')
client = MongoClient()
db = client.twitterdb

# Tweet Functions

In [135]:
def print_tweet(tweet):
    """
    Print text for a tweet
    """
    print(tweet['text'])


def store_tweet(tweet, collection):
    """
    Store tweet in MongoDB
    """
    db[collection].insert_one(tweet)


def limit_handled(cursor):
    """
    Code taken from tweepy to deal with rate limit issues
    """
    while True:
        try:
            yield cursor.next()
        except tweepy.RateLimitError:
            print("Rate limit exceeded. Sleeping ...")
            time.sleep(15 * 60)


# Donald Trump Twitter

In [120]:
user = api.get_user('realDonaldTrump')

In [144]:
# Get trump tweets
n = 100  # number of tweets to get

for tweet in limit_handled(tweepy.Cursor(
    api.user_timeline, id='realDonaldTrump', count=n).items(n)):
    
    store_tweet(tweet=tweet._json, collection='trump_tweets')

In [149]:
# Get trump retweets
n = 100  # number of retweets to get

for tweet in db.trump_tweets.find():
    retweets = api.retweets(tweet['id_str'], count=n)
    
    for retweet in retweets:
        retweet = retweet._json
        retweet['trump_tweet_mongo_id'] = tweet['_id']
        
        store_tweet(tweet=retweet, 
                    collection="trump_retweets_" + str(tweet['_id']))

RateLimitError: [{'message': 'Rate limit exceeded', 'code': 88}]

In [150]:
# Export MongoDB
!mongodump --db twitterdb --gzip --out "../data"

2017-10-31T17:23:52.186-0700	writing twitterdb.trump_retweets_59f90fc7a3b98c535ef358f3 to 
2017-10-31T17:23:52.187-0700	writing twitterdb.trump_retweets_59f90fc7a3b98c535ef358fa to 
2017-10-31T17:23:52.187-0700	writing twitterdb.trump_retweets_59f90fc7a3b98c535ef358fc to 
2017-10-31T17:23:52.187-0700	writing twitterdb.trump_retweets_59f90fc7a3b98c535ef35902 to 
2017-10-31T17:23:52.208-0700	done dumping twitterdb.trump_retweets_59f90fc7a3b98c535ef358fa (189 documents)
2017-10-31T17:23:52.208-0700	writing twitterdb.trump_retweets_59f90fc7a3b98c535ef358fe to 
2017-10-31T17:23:52.210-0700	done dumping twitterdb.trump_retweets_59f90fc7a3b98c535ef35902 (186 documents)
2017-10-31T17:23:52.210-0700	writing twitterdb.trump_retweets_59f90fc7a3b98c535ef358f2 to 
2017-10-31T17:23:52.210-0700	done dumping twitterdb.trump_retweets_59f90fc7a3b98c535ef358fc (188 documents)
2017-10-31T17:23:52.210-0700	writing twitterdb.trump_retweets_59f90fc7a3b98c535ef358fb to 
2017-10-31T17:23:52.222-0700	done dumpi